<a href="https://colab.research.google.com/github/AG-2002/Customer-Segmentation-and-Prediction-of-Term-Deposit-Subscriptions-in-Bank-Marketing/blob/main/Bank_Data(Model_fitting(DecisionTree%2CRandomForest)).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('bank_data_cleaned.csv')
df

,age,month,day_of_week,duration,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,poutcome_nonexistent,poutcome_success,pdays_new_recently contacted,pdays_new_sometimes ago,y
0,56,5,1,261,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
1,57,5,1,149,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
2,37,5,1,226,1,0,1.1,93.994,-36.4,4.857,...,0,1,0,0,1,1,0,0,0,0
3,40,5,1,151,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
4,56,5,1,307,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41171,73,11,5,334,1,0,-1.1,94.767,-50.8,1.028,...,0,1,0,0,0,1,0,0,0,1
41172,46,11,5,383,1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,0,0
41173,56,11,5,189,2,0,-1.1,94.767,-50.8,1.028,...,0,1,0,0,0,1,0,0,0,0
41174,44,11,5,442,1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,0,1


# **----------  DECISION TREE ----------**

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [5]:
X = df.drop(columns='y')
y = df['y']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 15],
    'class_weight': [ {0: 1, 1: 5}, {0: 1, 1: 7.5}, {0: 1, 1: 10}]
}

In [24]:
grid_dt = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [{0: 1, 1: 5}, {0: 1, 1: 7.5},
                                          {0: 1, 1: 10}],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [5, 10, 15, 20],
                         'min_samples_split': [2, 5, 10, 15],
                         'splitter': ['best', 'random']})

In [25]:
grid_dt.best_params_

{'class_weight': {0: 1, 1: 5},
 'criterion': 'gini',
 'max_depth': 20,
 'min_samples_split': 2,
 'splitter': 'best'}

In [26]:
dt = DecisionTreeClassifier(criterion= 'gini',
 max_depth= 5,
 min_samples_split= 2,
 splitter= 'best',
 class_weight= {0: 1, 1: 5})

In [27]:
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight={0: 1, 1: 5}, max_depth=5)

In [29]:
# Predict and evaluate
y_pred = dt.predict(X_test)
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

 Accuracy: 0.8519912578921807
[[6186 1122]
 [  97  831]]

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.85      0.91      7308
           1       0.43      0.90      0.58       928

    accuracy                           0.85      8236
   macro avg       0.71      0.87      0.74      8236
weighted avg       0.92      0.85      0.87      8236



# **-----------  RANDOM FOREST  ----------**

In [34]:
param_grid1 = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

In [37]:
grid_dt1 = GridSearchCV(RandomForestClassifier(), param_grid1)
grid_dt1.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]})

In [38]:
grid_dt1.best_params_

{'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200}

In [39]:
rf = RandomForestClassifier(n_estimators= 200,
                                       bootstrap=True,
                                       oob_score=True,
                                       max_samples=0.7,
                                       max_depth= 20,
                                       min_samples_split= 2,
                                       min_samples_leaf= 2,
                                       max_features= 'sqrt',
                                       class_weight='balanced')

In [40]:
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=20, max_samples=0.7,
                       min_samples_leaf=2, n_estimators=200, oob_score=True)

In [41]:
# Predict and evaluate
y_pred = rf.predict(X_test)
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

 Accuracy: 0.9071151044196212
[[6751  557]
 [ 208  720]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.92      0.95      7308
           1       0.56      0.78      0.65       928

    accuracy                           0.91      8236
   macro avg       0.77      0.85      0.80      8236
weighted avg       0.92      0.91      0.91      8236

